Reduce bias from vector, using algorithm from [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520) also help taken from coursera.org course Sequence Models(week2) by deeplearning.ai.

In [1]:
import numpy as np
import pickle

In [2]:
# load the vocabulary
with open('dict_embed_weights/en_dict.txt', 'rb') as f:
    en_dict = pickle.load(f)
with open('dict_embed_weights/bn/bn_dict.txt', 'rb') as f:
    bn_dict = pickle.load(f)

In [3]:
def cosine_similarity(vector1, vector2):
    # vector1 and vector2 are numpy array of similar shape
    # cosine similarity of two vector will be
    # dot product of two vector divided by muliplication of those vectors L2 norm
    
    # calculate L2 norm
    vector1_l2 = np.sqrt(np.sum(np.square(vector1)))
    vector2_l2 = np.sqrt(np.sum(np.square(vector2)))
    
    # calculate similarity
    similarity = np.dot(vector1, vector2) / (vector1_l2 * vector2_l2)
    
    return similarity

In [4]:
# let's print some similarities
print('Cosine_similarity(brother, sister) = ', cosine_similarity(en_dict['brother'], en_dict['sister']))
print('Cosine_similarity(ball, mango) = ', cosine_similarity(en_dict['ball'], en_dict['mango']))
print('Cosine_similarity(increase - decrease, negative - positive) = ', cosine_similarity((en_dict['increase']-en_dict['decrease']), (en_dict['negative']-en_dict['positive'])))
print('Cosine_similarity(brother - boy, sister - girl)', cosine_similarity((en_dict['brother']-en_dict['boy']), (en_dict['sister']-en_dict['girl'])))

Cosine_similarity(brother, sister) =  0.594084956538147
Cosine_similarity(ball, mango) =  0.001340670005048851
Cosine_similarity(increase - decrease, negative - positive) =  -0.14973426149530344
Cosine_similarity(brother - boy, sister - girl) 0.7199552431856013


In [5]:
def word_analogy(word1, word2, word3, vocab):
    # word1-word2 -> word3-?(word4)
    # word1, word2, word3 are word
    
    # choose the corresponding word vector from vocabulary
    word1_v = vocab[word1]
    word2_v = vocab[word2]
    word3_v = vocab[word3]
    
    # load the words from vocabulary
    words_in_dict = vocab.keys()
    input_words = [word1, word2, word3]
    cos_similarity = -10
    word4 = None  # keep the best suitable word
    for word in words_in_dict:
        
        # for skip the inputs word
        if word in input_words:
            continue
        
        # calculate cosine_similarity
        similarity = cosine_similarity((word1_v-word2_v), (word3_v-vocab[word]))
        
        if cos_similarity < similarity:
            cos_similarity = similarity
            word4 = word
    
    # return word with higher similarity
    return word4

In [6]:
# find out the 4th word for the following
print('man-woman -> grandfather-', word_analogy('man', 'woman', 'grandfather', en_dict))
print('boy-girl -> brother-', word_analogy('boy', 'girl', 'brother', en_dict))
print('bangladesh-bangladeshi -> australia-', word_analogy('bangladesh', 'bangladeshi', 'australia', en_dict))
print('big-bigger -> tall-', word_analogy('big', 'bigger', 'tall', en_dict))

man-woman -> grandfather- grandmother
boy-girl -> brother- sister
bangladesh-bangladeshi -> australia- australian
big-bigger -> tall- taller


In [7]:
# let's find a vector which roughly encodes the gender idea
#g1 = en_dict['grandmother'] - en_dict['grandfather']
g2 = en_dict['woman'] - en_dict['man']
g3 = en_dict['mother'] -  en_dict['father']
#g4 = en_dict['girl'] - en_dict['boy']
c_gender = (g2 + g3) / 2

In [8]:
# seeing how calculated vector reflect with girl and boy names

# some randomly taken girl and boy names
names = ['james', 'william', 'john', 'ronaldo', 'olivia', 'emma', 'mia', 'isabella']
for name in names:
    print(name + ' : ', cosine_similarity(c_gender, en_dict[name]))

james :  -0.20298910002941703
william :  -0.189571638803583
john :  -0.2551546186934181
ronaldo :  -0.08791088909446129
olivia :  0.17366318175830034
emma :  0.21878416043848326
mia :  0.16786658942395405
isabella :  0.22117549290440713


In [9]:
# some random words which may or may not connect with gender
other_words = ['ring', 'bracelet', 'guns', 'lipstick', 'eyeglass', 'science', 'tree', 'literature', 'fashion', 'earth']

# some random professions
professions = ['writer', 'doctor', 'engineer', 'singer', 'pilot', 'receptionist', 'teacher', 'designer', 'knight']

print('Similarities between some words : ')
for word in other_words:
    print(word + ' : ', cosine_similarity(c_gender, en_dict[word]))

print('\nSimilarities between professions : ')
for word in professions:
    print(word + ' : ', cosine_similarity(c_gender, en_dict[word]))

Similarities between some words : 
ring :  -0.008046233062665014
bracelet :  0.10745294737223761
guns :  -0.07413901847806628
lipstick :  0.2579265516199865
eyeglass :  0.07436438121281154
science :  -0.043420585010653066
tree :  0.03743565451348173
literature :  0.05052154843581333
fashion :  0.15638154525415773
earth :  -0.016548570863480386

Similarities between professions : 
writer :  0.0379288445271604
doctor :  0.08424000526451889
engineer :  -0.17525582575897017
singer :  0.18314793413331615
pilot :  -0.053400816522660384
receptionist :  0.2679675972384471
teacher :  0.14454457496574427
designer :  0.06791301366103841
knight :  -0.07423116178263871


From previous similarity output we can see that there are some result which we did expect.
But if you notice clearly then bracelet, teacher some other word does not give the result we expect. It may occur for gender biasing.

In [10]:
# neutralize non-gender specific word
def remove_bias(word_vector, c_gender):
    # word_vector a word vector which you want to neutralize for gender
    # c_gender a vector which roughly encodes the gender idea
    
    # calculate l2 norm square
    c_gender_l2_square = np.sum(np.square(c_gender))
    
    # calculate bias in word vector
    bias = (np.dot(word_vector, c_gender) / c_gender_l2_square) * c_gender
    
    # remove bias from word vector
    word_vector_debias = word_vector - bias
    
    return word_vector_debias

In [11]:
# list of word for debias
list_of_word = ['bracelet', 'fashion', 'teacher', 'receptionist']

for word in list_of_word:
    print('Similarity between ' + word + ', c_gender before remove bias : ', cosine_similarity(c_gender, en_dict[word]))
    word_vector_debias = remove_bias(en_dict[word], c_gender)
    print('Similarity between ' + word + ', c_gender after remove bias : ', cosine_similarity(c_gender, word_vector_debias))
    print('\n')

Similarity between bracelet, c_gender before remove bias :  0.10745294737223761
Similarity between bracelet, c_gender after remove bias :  -7.669631344943666e-18


Similarity between fashion, c_gender before remove bias :  0.15638154525415773
Similarity between fashion, c_gender after remove bias :  2.145132698151772e-18


Similarity between teacher, c_gender before remove bias :  0.14454457496574427
Similarity between teacher, c_gender after remove bias :  -1.7518980905370778e-17


Similarity between receptionist, c_gender before remove bias :  0.2679675972384471
Similarity between receptionist, c_gender after remove bias :  1.0287342325867356e-17




In [12]:
# neutralize distance for gender specific word
def equalize_words_distance(word_pair_vector, bias_vector):
    # bias_vector a vector from which distance will be equal
    # word_pair_vector pair of word vectors
    
    word1_v, word2_v = word_pair_vector
    
    # calculate mean
    mean = (word1_v + word2_v) / 2
    
    # calculate L2 norm square for bias_vector
    norm_b = np.sum(np.square(bias_vector))
    
    # calculate mean bias
    mean_bias = (np.dot(mean, bias_vector) / norm_b) * bias_vector
    mean_debias = mean - mean_bias
    
    # calculate word1_v bias
    word1_v_bias = (np.dot(word1_v, bias_vector) / norm_b) * bias_vector
    
    # calculate word2_v bias
    word2_v_bias = (np.dot(word2_v, bias_vector) / norm_b) * bias_vector
    
    # calculate L2 norm square for mean_debias
    norm_md = np.sum(np.square(mean_debias))
    
    # adjust bias with mean_bias
    word1_v_adj = np.sqrt(np.abs(1-norm_md)) * ((word1_v_bias-mean_bias) / np.sqrt(np.sum(np.square((word1_v-mean_debias)-mean_bias))))
    word2_v_adj = np.sqrt(np.abs(1-norm_md)) * ((word2_v_bias-mean_bias) / np.sqrt(np.sum(np.square((word2_v-mean_debias)-mean_bias))))
    
    # calculate final words vector
    final_word1_v = word1_v_adj + mean_debias
    final_word2_v = word2_v_adj + mean_debias
    
    return (final_word1_v, final_word2_v)

In [13]:
print('Similarity between man, c_gender before equalize distance : ', cosine_similarity(c_gender, en_dict['man']))
print('Similarity between woman, c_gender before equalize distance : ', cosine_similarity(c_gender, en_dict['woman']))
final_word1_v, final_word2_v = equalize_words_distance((en_dict['man'], en_dict['woman']), c_gender)
print('Similarity between man, c_gender after equalize distance : ', cosine_similarity(c_gender, final_word1_v))
print('Similarity between woman, c_gender after equalize distance : ', cosine_similarity(c_gender, final_word2_v))
print('\n')
print('Similarity between grandmother, c_gender before equalize distance : ', cosine_similarity(c_gender, en_dict['grandmother']))
print('Similarity between grandfather, c_gender before equalize distance : ', cosine_similarity(c_gender, en_dict['grandfather']))
final_word1_v, final_word2_v = equalize_words_distance((en_dict['grandmother'], en_dict['grandfather']), c_gender)
print('Similarity between grandmother, c_gender after equalize distance : ', cosine_similarity(c_gender, final_word1_v))
print('Similarity between grandfather, c_gender after equalize distance : ', cosine_similarity(c_gender, final_word2_v))

Similarity between man, c_gender before equalize distance :  -0.22983910313161754
Similarity between woman, c_gender before equalize distance :  0.4673801734130888
Similarity between man, c_gender after equalize distance :  -0.6658761946166748
Similarity between woman, c_gender after equalize distance :  0.6658761946166747


Similarity between grandmother, c_gender before equalize distance :  0.3371162810641586
Similarity between grandfather, c_gender before equalize distance :  -0.20021611604548606
Similarity between grandmother, c_gender after equalize distance :  0.5669759058114134
Similarity between grandfather, c_gender after equalize distance :  -0.5669759058114134
